In [1]:
import pandas as pd
import numpy as np
import mxnet as mx
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
train = pd.read_json('./data/train/train.json')

In [3]:
print(train.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [4]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1604 entries, 0 to 1603
Data columns (total 5 columns):
band_1        1604 non-null object
band_2        1604 non-null object
id            1604 non-null object
inc_angle     1604 non-null object
is_iceberg    1604 non-null int64
dtypes: int64(1), object(4)
memory usage: 75.2+ KB
None


In [5]:
print(train.is_iceberg.value_counts())

0    851
1    753
Name: is_iceberg, dtype: int64


In [6]:
print(train.describe())

        is_iceberg
count  1604.000000
mean      0.469451
std       0.499222
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000


In [7]:
def build_x(train_no_test=True):
    if train_no_test:
        x_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
        x_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])
    else:
        x_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
        x_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])
    
    res = np.concatenate([x_band_1[:, :, :, np.newaxis], x_band_2[:, :, :, np.newaxis], ((x_band_1+x_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
    
    return res.transpose(0, 3, 1, 2)

In [8]:
x_train = build_x()
print(x_train.shape)

(1604, 3, 75, 75)


In [9]:
import plotly.offline as py
import plotly.graph_objs as go

In [10]:
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):
    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [11]:
plotmy3d(x_train[2,0,:,:], 'iceberg')

In [12]:
plotmy3d(x_train[0,0,:,:], 'ship')

In [13]:
plotmy3d(x_train[2,1,:,:], 'iceberg')

In [14]:
plotmy3d(x_train[0,1,:,:], 'ship')

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
y_train = pd.get_dummies(train.is_iceberg)

In [17]:
print(y_train.head())

   0  1
0  1  0
1  1  0
2  0  1
3  1  0
4  1  0


In [18]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

In [19]:
# should be optional as iterator can take np array as input
x_train = mx.ndarray.array(x_train)
x_valid = mx.ndarray.array(x_valid)
y_train = mx.ndarray.array(y_train)
y_valid = mx.ndarray.array(y_valid)

In [20]:
print(x_train.shape)
print(type(x_train))
print(y_train.shape)

(1283, 3, 75, 75)
<class 'mxnet.ndarray.ndarray.NDArray'>
(1283, 2)


In [21]:
batch_size = 50
train_iter = mx.io.NDArrayIter(data=x_train, label=y_train, batch_size=batch_size, shuffle=True)
valid_iter = mx.io.NDArrayIter(data=x_valid, label=y_valid, batch_size=batch_size)

In [22]:
data = mx.sym.Variable('data')
# convolution layer 1
conv1 = mx.sym.Convolution(data=data, kernel=(3,3), num_filter=64)
relu1 = mx.sym.Activation(data=conv1, act_type='relu')
pool1 = mx.sym.Pooling(data=relu1, pool_type='max', kernel=(3,3), stride=(2,2))

# convolution layer 2
conv2 = mx.sym.Convolution(data=pool1, kernel=(3,3), num_filter=128)
relu2 = mx.sym.Activation(data=conv2, act_type='relu')
pool2 = mx.sym.Pooling(data=relu2, pool_type='max', kernel=(2,2), stride=(2,2))

# convolution layer 3
conv3 = mx.sym.Convolution(data=pool2, kernel=(3,3), num_filter=128)
relu3 = mx.sym.Activation(data=conv3, act_type='relu')
pool3 = mx.sym.Pooling(data=relu3, pool_type='max', kernel=(2,2), stride=(2,2))

# convolution layer 4
conv4 = mx.sym.Convolution(data=pool3, kernel=(3,3), num_filter=64)
relu4 = mx.sym.Activation(data=conv4, act_type='relu')
pool4 = mx.sym.Pooling(data=relu4, pool_type='max', kernel=(2,2), stride=(2,2))

# flatten data
flatten = mx.sym.flatten(pool4)

# fully connected layers
# hidden layer 1
hidden1 = mx.sym.FullyConnected(data=flatten, num_hidden=512)
relu5 = mx.sym.Activation(data=hidden1, act_type='relu')

# hidden layer 2
hidden2 = mx.sym.FullyConnected(data=relu5, num_hidden=256)
relu6 = mx.sym.Activation(data=hidden2, act_type='relu')

# decision layer
hidden3 = mx.sym.FullyConnected(data=relu6, num_hidden=2)
out = mx.sym.SoftmaxOutput(data=hidden3, name='softmax')

In [23]:
print(out.list_arguments())

['data', 'convolution0_weight', 'convolution0_bias', 'convolution1_weight', 'convolution1_bias', 'convolution2_weight', 'convolution2_bias', 'convolution3_weight', 'convolution3_bias', 'fullyconnected0_weight', 'fullyconnected0_bias', 'fullyconnected1_weight', 'fullyconnected1_bias', 'fullyconnected2_weight', 'fullyconnected2_bias', 'softmax_label']


In [24]:
model = mx.mod.Module(symbol=out, context=mx.gpu())

In [25]:
model.fit(train_iter,
          eval_data=valid_iter,
          optimizer='adam',
          optimizer_params={'learning_rate':0.1},
          eval_metric='acc',
          initializer=mx.initializer.Xavier(),
          batch_end_callback=mx.callback.Speedometer(1, 10),
          num_epoch=20)

In [26]:
acc = mx.metric.Accuracy()
model.score(valid_iter, acc)
print(acc)

EvalMetric: {'accuracy': 0.5}
